In [47]:
import tensorflow as tf
import numpy as np 
import pandas as pd
rows = 10
net = tf.keras.layers.Dense(2) # 一个隐藏层，一个神经元
net.build((rows, 3)) # 每个训练数据有1个特征
print("net.w:", net.kernel) # 参数个数
print("net.b:", net.bias) # 和Dense数一样


net.w: <tf.Variable 'kernel:0' shape=(3, 2) dtype=float32, numpy=
array([[-0.3890481 ,  0.39112937],
       [ 0.8697113 ,  0.13425934],
       [ 0.13820052,  0.8496635 ]], dtype=float32)>
net.b: <tf.Variable 'bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>
